In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_guichu_tiaojiao=pd.DataFrame(columns=columns)
channel_guichu_tiaojiao

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[10]/div/a/span').click()  #选择频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[10]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'鬼畜'

In [9]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').click()  #选择子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[2]/a').text
sub_channel

'鬼畜调教'

In [11]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [12]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_guichu_tiaojiao.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 
    


2020-12-16 20:14:04.949579


<ipython-input-12-91a4747a5db4>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-91a4747a5db4>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-12-91a4747a5db4>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1AJ411j7mv视频属性的抓取
BV号为1AJ411j7mv的视频属性抓取结束


<ipython-input-12-91a4747a5db4>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1r7411k7eN视频属性的抓取
BV号为1r7411k7eN的视频属性抓取结束
开始BV号为1yJ411h7qW视频属性的抓取
BV号为1yJ411h7qW的视频属性抓取结束
开始BV号为17J411b7WV视频属性的抓取
BV号为17J411b7WV的视频属性抓取结束
开始BV号为1w7411v7xs视频属性的抓取
BV号为1w7411v7xs的视频属性抓取结束
开始BV号为177411e7gh视频属性的抓取
BV号为177411e7gh的视频属性抓取结束
开始BV号为1F7411C71w视频属性的抓取
BV号为1F7411C71w的视频属性抓取结束
开始BV号为1eJ411p71G视频属性的抓取
BV号为1eJ411p71G的视频属性抓取结束
开始BV号为1m7411i7ZY视频属性的抓取
BV号为1m7411i7ZY的视频属性抓取结束
开始BV号为1xJ411J7Ue视频属性的抓取
BV号为1xJ411J7Ue的视频属性抓取结束
开始BV号为137411e7pB视频属性的抓取
BV号为137411e7pB的视频属性抓取结束
开始BV号为1VJ411G73Z视频属性的抓取
BV号为1VJ411G73Z的视频属性抓取结束
开始BV号为1y7411Y7fg视频属性的抓取
BV号为1y7411Y7fg的视频属性抓取结束
开始BV号为1B7411e7eA视频属性的抓取
BV号为1B7411e7eA的视频属性抓取结束
开始BV号为1zJ411j7a1视频属性的抓取
BV号为1zJ411j7a1的视频属性抓取结束
开始BV号为15J411n7KE视频属性的抓取
BV号为15J411n7KE的视频属性抓取结束
开始BV号为1KJ411n7xB视频属性的抓取
BV号为1KJ411n7xB的视频属性抓取结束
开始BV号为1M7411r7vb视频属性的抓取
BV号为1M7411r7vb的视频属性抓取结束
开始BV号为1tJ411n7PG视频属性的抓取
BV号为1tJ411n7PG的视频属性抓取结束
开始BV号为1iJ411G7z6视频属性的抓取
BV号为1iJ411G7z6的视频属性抓取结束
开始BV号为1BJ411H7JC视频属性的抓取
BV号为1BJ411H7JC的视频属性抓取结束
开始BV号为1p7411z7Le视频属性的抓取
BV号为1p7411z7Le的视

BV号为1c7411P73X的视频属性抓取结束
开始BV号为197411J7n7视频属性的抓取
BV号为197411J7n7的视频属性抓取结束
开始BV号为1r7411T78d视频属性的抓取
BV号为1r7411T78d的视频属性抓取结束
开始BV号为1Bj411f7eY视频属性的抓取
BV号为1Bj411f7eY的视频属性抓取结束
开始BV号为1P7411T7ym视频属性的抓取
BV号为1P7411T7ym的视频属性抓取结束
开始BV号为197411n7av视频属性的抓取
BV号为197411n7av的视频属性抓取结束
开始BV号为1P7411t76p视频属性的抓取
BV号为1P7411t76p的视频属性抓取结束
开始BV号为1c7411E7g4视频属性的抓取
BV号为1c7411E7g4的视频属性抓取结束
开始BV号为1g7411T75b视频属性的抓取
BV号为1g7411T75b的视频属性抓取结束
开始BV号为167411t78f视频属性的抓取
BV号为167411t78f的视频属性抓取结束
开始BV号为1s7411w7Qk视频属性的抓取
BV号为1s7411w7Qk的视频属性抓取结束
开始BV号为1G7411J7KT视频属性的抓取
BV号为1G7411J7KT的视频属性抓取结束
开始BV号为1n7411W7hV视频属性的抓取
BV号为1n7411W7hV的视频属性抓取结束
开始BV号为1y7411t7Qq视频属性的抓取
BV号为1y7411t7Qq的视频属性抓取结束
开始BV号为117411W7z3视频属性的抓取
BV号为117411W7z3的视频属性抓取结束
开始BV号为17741137c1视频属性的抓取
BV号为17741137c1的视频属性抓取结束
开始BV号为1x7411V7No视频属性的抓取
BV号为1x7411V7No的视频属性抓取结束
开始BV号为167411x76G视频属性的抓取
BV号为167411x76G的视频属性抓取结束
开始BV号为1x741177dT视频属性的抓取
BV号为1x741177dT的视频属性抓取结束
开始BV号为1b7411j71n视频属性的抓取
BV号为1b7411j71n的视频属性抓取结束
开始BV号为1i7411c7UN视频属性的抓取
BV号为1i7411c7UN的视频属性抓取结束
开始BV号为1t7411n7j6

BV号为13e411472T的视频属性抓取结束
开始BV号为1Sk4y1R77y视频属性的抓取
BV号为1Sk4y1R77y的视频属性抓取结束
开始BV号为1VK4y1k7rq视频属性的抓取
BV号为1VK4y1k7rq的视频属性抓取结束
开始BV号为1Fp4y197J5视频属性的抓取
BV号为1Fp4y197J5的视频属性抓取结束
开始BV号为1RK411L7hw视频属性的抓取
BV号为1RK411L7hw的视频属性抓取结束
开始BV号为1FQ4y1T7zD视频属性的抓取
BV号为1FQ4y1T7zD的视频属性抓取结束
开始BV号为1NQ4y1T7ko视频属性的抓取
BV号为1NQ4y1T7ko的视频属性抓取结束
开始BV号为1wf4y1S79G视频属性的抓取
BV号为1wf4y1S79G的视频属性抓取结束
开始BV号为13f4y1S7tt视频属性的抓取
BV号为13f4y1S7tt的视频属性抓取结束
开始BV号为1n64y1u7X7视频属性的抓取
BV号为1n64y1u7X7的视频属性抓取结束
开始BV号为1at4y1U7Zd视频属性的抓取
BV号为1at4y1U7Zd的视频属性抓取结束
开始BV号为1gi4y1t7nZ视频属性的抓取
BV号为1gi4y1t7nZ的视频属性抓取结束
开始BV号为1ip4y1C7fR视频属性的抓取
BV号为1ip4y1C7fR的视频属性抓取结束
开始BV号为1NK411j7Kh视频属性的抓取
BV号为1NK411j7Kh的视频属性抓取结束
开始BV号为1Sg4y187qo视频属性的抓取
BV号为1Sg4y187qo的视频属性抓取结束
开始BV号为1uT4y1V7cQ视频属性的抓取
BV号为1uT4y1V7cQ的视频属性抓取结束
开始BV号为11i4y1b7Mu视频属性的抓取
BV号为11i4y1b7Mu的视频属性抓取结束
开始BV号为1Ep4y1y7na视频属性的抓取
BV号为1Ep4y1y7na的视频属性抓取结束
开始BV号为1EQ4y1T7Tv视频属性的抓取
BV号为1EQ4y1T7Tv的视频属性抓取结束
开始BV号为1Ti4y1b7fu视频属性的抓取
BV号为1Ti4y1b7fu的视频属性抓取结束
开始BV号为12A41147DA视频属性的抓取
BV号为12A41147DA的视频属性抓取结束
开始BV号为15K4y1k7Fu

开始BV号为1Bz411e7h8视频属性的抓取
BV号为1Bz411e7h8的视频属性抓取结束
开始BV号为1Bg4y1q7Nt视频属性的抓取
BV号为1Bg4y1q7Nt的视频属性抓取结束
开始BV号为1ca4y1e7DP视频属性的抓取
BV号为1ca4y1e7DP的视频属性抓取结束
开始BV号为1mt4y1977Y视频属性的抓取
BV号为1mt4y1977Y的视频属性抓取结束
开始BV号为1Wg4y1q7nE视频属性的抓取
BV号为1Wg4y1q7nE的视频属性抓取结束
开始BV号为1M5411W78f视频属性的抓取
BV号为1M5411W78f的视频属性抓取结束
开始BV号为175411Y7xA视频属性的抓取
BV号为175411Y7xA的视频属性抓取结束
开始BV号为1DK4y1x7cR视频属性的抓取
BV号为1DK4y1x7cR的视频属性抓取结束
开始BV号为19t4y1X7jD视频属性的抓取
BV号为19t4y1X7jD的视频属性抓取结束
开始BV号为1ia4y1v7V1视频属性的抓取
BV号为1ia4y1v7V1的视频属性抓取结束
开始BV号为1Ug4y1i7WJ视频属性的抓取
BV号为1Ug4y1i7WJ的视频属性抓取结束
开始BV号为16k4y1z7NL视频属性的抓取
BV号为16k4y1z7NL的视频属性抓取结束
开始BV号为1st4y197NL视频属性的抓取
BV号为1st4y197NL的视频属性抓取结束
开始BV号为1FC4y1a7jo视频属性的抓取
BV号为1FC4y1a7jo的视频属性抓取结束
开始BV号为1mQ4y1P7CQ视频属性的抓取
BV号为1mQ4y1P7CQ的视频属性抓取结束
开始BV号为1Sk4y1B73E视频属性的抓取
BV号为1Sk4y1B73E的视频属性抓取结束
开始BV号为1gT4y1J7xg视频属性的抓取
BV号为1gT4y1J7xg的视频属性抓取结束
开始BV号为1fA411q73d视频属性的抓取
BV号为1fA411q73d的视频属性抓取结束
开始BV号为1DK411n77s视频属性的抓取
BV号为1DK411n77s的视频属性抓取结束
开始BV号为12p4y1S7Xa视频属性的抓取
BV号为12p4y1S7Xa的视频属性抓取结束
开始BV号为1hD4y1Q7WK视频属性的抓取
BV号为1hD4y1Q7WK的视

BV号为1Ev411q724的视频属性抓取结束
开始BV号为1FK411n7yx视频属性的抓取
BV号为1FK411n7yx的视频属性抓取结束
开始BV号为1tk4y1q7xf视频属性的抓取
BV号为1tk4y1q7xf的视频属性抓取结束
开始BV号为1fV411B7cq视频属性的抓取
BV号为1fV411B7cq的视频属性抓取结束
开始BV号为1pv411q7Uv视频属性的抓取
BV号为1pv411q7Uv的视频属性抓取结束
开始BV号为1w54y1S7Yx视频属性的抓取
BV号为1w54y1S7Yx的视频属性抓取结束
开始BV号为13z4y1D79u视频属性的抓取
BV号为13z4y1D79u的视频属性抓取结束
开始BV号为1JC4y1h7yQ视频属性的抓取
BV号为1JC4y1h7yQ的视频属性抓取结束
开始BV号为18p4y1S7k6视频属性的抓取
BV号为18p4y1S7k6的视频属性抓取结束
开始BV号为1BK411n7mW视频属性的抓取
BV号为1BK411n7mW的视频属性抓取结束
开始BV号为1Ah411Z7UY视频属性的抓取
BV号为1Ah411Z7UY的视频属性抓取结束
开始BV号为1Gi4y1G7RV视频属性的抓取
BV号为1Gi4y1G7RV的视频属性抓取结束
开始BV号为1d54y1U7E2视频属性的抓取
BV号为1d54y1U7E2的视频属性抓取结束
开始BV号为1N5411e7vn视频属性的抓取
BV号为1N5411e7vn的视频属性抓取结束
开始BV号为11p4y1U7qc视频属性的抓取
BV号为11p4y1U7qc的视频属性抓取结束
开始BV号为1CA411e7J1视频属性的抓取
BV号为1CA411e7J1的视频属性抓取结束
开始BV号为16g4y1v7VS视频属性的抓取
BV号为16g4y1v7VS的视频属性抓取结束
开始BV号为1Pg4y1i766视频属性的抓取
BV号为1Pg4y1i766的视频属性抓取结束
开始BV号为1rC4y1h7ib视频属性的抓取
BV号为1rC4y1h7ib的视频属性抓取结束
开始BV号为1Kh411Z7Ry视频属性的抓取
BV号为1Kh411Z7Ry的视频属性抓取结束
开始BV号为1Ga4y1E7Qq视频属性的抓取
BV号为1Ga4y1E7Qq的视频属性抓取结束
开始BV号为1gV411U7pC

开始BV号为1Li4y1u7HY视频属性的抓取
BV号为1Li4y1u7HY的视频属性抓取结束
开始BV号为1zt4y1S7NV视频属性的抓取
BV号为1zt4y1S7NV的视频属性抓取结束
开始BV号为1H54y1v7JW视频属性的抓取
BV号为1H54y1v7JW的视频属性抓取结束
开始BV号为1F54y1y7ix视频属性的抓取
BV号为1F54y1y7ix的视频属性抓取结束
开始BV号为1kT4y1c7Lw视频属性的抓取
BV号为1kT4y1c7Lw的视频属性抓取结束
开始BV号为1La4y1E7oe视频属性的抓取
BV号为1La4y1E7oe的视频属性抓取结束
开始BV号为1tT4y1c7RN视频属性的抓取
BV号为1tT4y1c7RN的视频属性抓取结束
开始BV号为1Ct4y1i7Fe视频属性的抓取
BV号为1Ct4y1i7Fe的视频属性抓取结束
开始BV号为1wf4y1X7Rk视频属性的抓取
BV号为1wf4y1X7Rk的视频属性抓取结束
开始BV号为1EZ4y157LD视频属性的抓取
BV号为1EZ4y157LD的视频属性抓取结束
开始BV号为1Ui4y1M71D视频属性的抓取
BV号为1Ui4y1M71D的视频属性抓取结束
开始BV号为1yy4y1C7Mt视频属性的抓取
BV号为1yy4y1C7Mt的视频属性抓取结束
开始BV号为1Uk4y1y7B5视频属性的抓取
BV号为1Uk4y1y7B5的视频属性抓取结束
开始BV号为1yV4112726视频属性的抓取
BV号为1yV4112726的视频属性抓取结束
开始BV号为1ZC4y1t7LE视频属性的抓取
BV号为1ZC4y1t7LE的视频属性抓取结束
开始BV号为1Qi4y1M76h视频属性的抓取
BV号为1Qi4y1M76h的视频属性抓取结束
开始BV号为1nv411C7u3视频属性的抓取
BV号为1nv411C7u3的视频属性抓取结束
开始BV号为19t4y1i7QZ视频属性的抓取
BV号为19t4y1i7QZ的视频属性抓取结束
开始BV号为1WK4y1a716视频属性的抓取
BV号为1WK4y1a716的视频属性抓取结束
开始BV号为1Rf4y1S7mb视频属性的抓取
BV号为1Rf4y1S7mb的视频属性抓取结束
开始BV号为1J541187FJ视频属性的抓取
BV号为1J541187FJ的视

In [14]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-14-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [13]:
channel_guichu_tiaojiao.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [14]:
channel_guichu_tiaojiao.to_json('channel_guichu_tiaojiao.json')

In [17]:
pd.set_option('display.max_rows', None)

In [18]:
channel_youxi_danji[channel_youxi_danji['时间']=='4月']

,时间,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1h54y1d7Qq,4月,鹤吱菌,个人认证,191.3万,81,【师生对线】这网课上得就尼玛离谱！（第五集）,游戏,单机游戏,活动作品,自制,"[配音, 守望先锋, 独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, 电子竞技, 配音大...",13:00,953.7万,114.2万,115.2万,42.5万,8.9万,25302,18.8万,否
1L64y1M7Dp,4月,老番茄,个人认证,1360.1万,348,【老番茄】史上最骚骑士,游戏,单机游戏,活动作品,自制,"[骑马与砍杀, 搞笑, 游戏, 单机游戏, 老番茄, 骑马与砍杀2, 游戏练级挑战, 骑砍,...",15:45,902.3万,84.5万,61.7万,24.5万,2.4万,21336,9.7万,否
18K41157Tu,4月,老番茄,个人认证,1360.1万,348,宫斗大师老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 宫妃, 易次元]",13:55,739.6万,45.1万,9.3万,5.5万,1.3万,14189,10.6万,否
1PT4y1G7ME,4月,老番茄,个人认证,1360.1万,348,常胜将军老番茄,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄, shield wall]",14:54,726.8万,48.7万,12.9万,5.8万,7401,15180,7.0万,否
1S54y197uX,4月,徐大虾咯,个人认证,570.1万,224,家 里 有 矿 ！,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:18,714.5万,35.0万,8.8万,4.3万,4860,7738,6.7万,否
1ti4y1t7YM,4月,老番茄,个人认证,1360.1万,348,解说挑战！什么游戏都能解说？,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 老番茄]",08:19,711.2万,61.4万,16.7万,8.1万,9970,14368,10.6万,否
1Ai4y1b7ET,4月,敖厂长,个人认证,735.1万,348,花了一万多元终于见到最早开放世界游戏之一,游戏,单机游戏,普通视频,自制,"[游戏, 单机游戏, 游戏资讯, 敖厂长, 单机, 游戏杂谈, 游戏视频, 杂谈]",14:26,670.4万,112.6万,156.8万,34.9万,2.4万,15618,8.6万,否
1Up4y1C7PB,4月,丢鹏D1uuu,无,8.4万,20,年级第一：别TM挡着我上网课！！！,游戏,单机游戏,活动作品,自制,"[独立游戏, 搞笑, 单机游戏, 游戏, 单机游戏, bilibili新星计划, 新人, 恶...",05:38,654.0万,17.3万,1.5万,2.0万,1321,1952,4526,否
1wi4y187bu,4月,中国BOY超级大猩猩,个人认证,663.5万,1380,全员铁憨憨!,游戏,单机游戏,活动作品,自制,"[STEAM, 单机游戏, 游戏, 单机游戏, 花少北, 老番茄, 中国BOY, LexBu...",19:57,653.2万,42.0万,23.9万,14.2万,2.5万,13493,9.7万,否
13t4y1U7kQ,4月,徐大虾咯,个人认证,570.1万,224,德华的世界#2,游戏,单机游戏,普通视频,自制,"[我的世界, 单机游戏, 游戏, 单机游戏, 徐大虾咯, 徐大虾, MINECRAFT, M...",12:07,619.7万,35.1万,9.5万,4.4万,7033,7937,6.3万,否


In [12]:
channel_shuma_shexiang

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1gJ411n7hp,1月,2020-01-12 16:31:27,甜咖啡后味带苦,无,723,16,中 门 对 狙,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, 教程, 器材]",01:44,222.4万,9.7万,2.0万,1.4万,4980,3182,1571,否
1tJ41187w6,1月,2020-01-02 19:43:25,-LKs-,个人认证,165万,280,我找了11位知名UP主评价自己拍视频的设备丨器材终极选购指南,数码,摄影摄像,活动作品,...,"[相机, 数码, 摄影摄像, 数码, 何同学, 啊吗粽, 宝剑嫂, 自媒体, 大祥哥, 体验...",30:03,134.0万,10.7万,8.4万,5.8万,8907,3935,3.0万,否
1i7411i7yA,1月,2020-01-18 14:13:08,老蛙镜头,机构认证,2.6万,78,20秒的视频做了1个多月？揭秘模型世界的背后，准备好你的鸡皮疙瘩，51秒处别眨眼！——老蛙2...,数码,摄影摄像,普通视频,...,"[摄影, 短片, 数码, 摄影摄像, 数码, 创意, YOUTUBE, 镜头, 老蛙]",04:51,57.8万,4.4万,1.0万,1.6万,2043,641,983,否
1gJ411G7Cs,1月,2020-01-08 21:39:43,摄影师逝水,个人认证,0,84,一些实用且堪称神器的PS插件,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, VLOG, 后期, 教程, ps, 器材]",06:01,42.3万,3.3万,3.4万,8.6万,2605,1405,612,否
1KJ411e7nN,1月,2020-01-02 20:00:26,TenGuSan,个人认证,52.9万,76,如何提高审美？审美有无高低？审美是谁建立？,数码,摄影摄像,普通视频,...,"[校园vlog, 相机, 脱口秀, 摄影, 公开课, vlog, 科普, 数码, 摄影摄像,...",27:08,39.6万,4.8万,5.4万,3.4万,6217,1156,1.0万,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1cr4y1w7ig,10月,2020-10-30 20:35:37,巡天者叶梓颐,个人认证,8.6万,65,困难重重，国家天文签约摄影师遭遇拍摄滑铁卢？！【华为Mate40 Pro】,数码,摄影摄像,活动作品,...,"[vlog, 数码, 摄影摄像, 数码, 放心购机攻略, 敦煌, 星空, 手机摄影, 华为,...",11:43,4.3万,3051,1391,692,255,337,349,否
1Y5411L7TW,10月,2020-10-22 21:20:39,深蓝摄影评测,无,6518,30,富士X-S10相机测试——2020 apsc画幅性价比最强机,数码,摄影摄像,活动作品,...,"[相机, 摄影, 数码, 摄影摄像, 数码, 数码玩家训练营, 拍照, 测评, xs10, ...",10:20,4.3万,773,219,221,111,273,104,否
1c541177hR,10月,2020-10-10 13:10:50,HusseinsDrama,个人认证,50.3万,279,索尼（Sony）12-24mm F2.8 GM深度评测，只要买得起，它就是一颗完美的镜头！,数码,摄影摄像,普通视频,...,"[摄影, 数码, 摄影摄像, 数码, 评测, 超广角大光圈, 摄像, 镜头, 数码评测, 超...",21:23,4.1万,1469,443,297,123,250,561,否
1DA41177UY,10月,2020-10-15 11:00:15,DJI大疆创新,机构认证,15.2万,495,大疆旗舰稳定器 - DJI RS 2 正式登场,数码,摄影摄像,普通视频,...,"[相机, 摄影, 数码, 摄影摄像, 数码, 科技, 测评, 器材, 大疆, DJI, 稳定器]",02:26,4.1万,1565,197,334,640,171,110,否
